In [6]:
# Settings:
FILENAME = "C:/Users/Tristan/Projects/feasts.xlsx" # Change this to your filepath (supports .csv and .xlsx)
# File format: name, book, time, start line, end line, color (optional)

# For excel:
WORKSHEET = "feasts" # specify the name of the worksheet you want to be read.

# Here you can specify which colors are associated with each number specified in your excel sheet
# If your excel sheet lacks the color column, we will rotate through the colors listed below: 
COLOR_MAP = {
    1: "#c8c8c8",
    2: "gainsboro",
    3: "whitesmoke",
    4: "lightgray",
}
# Further information on colors can be found at: https://www.cssportal.com/css3-color-names/
# Note: be sure to write color names in all lowercase when typing them into the color map
# Grayscale colors: https://www.w3schools.com/colors/colors_shades.asp
# you can use nameless colors by specifying their hex value
# for example, by writing "#dcdcdc" rather than "gainsboro", you get the same color

# Here you can specify the names of the plots that are generated
BOOK_MAP = {
    1: "Book 1",
    2: "Book 2"
}

# Here you control the labelling of the Y-Axis:
Y_AXIS = {
    1: "Story Time",
    2: "Narrative Time"
}
   
# This controls the title of the X-Axis:
X_AXIS_TITLE = "Line Number"

# These control the resolution of the produced image:
# Set width and height to 0 to get image autosized to your window size
WIDTH = 2000
HEIGHT = 2000


# Here there be dragons...
import string
import plotly
import plotly.graph_objects as go
from openpyxl import load_workbook

LABELS = iter(string.ascii_lowercase)

class Scene():
    def __init__(self, name, book, time, start, end, color=None):
        self.name = name
        self.book = book
        self.time = time
        self.start = start
        self.end = end
        self.color = color
        self.label = ''
        
def create_scene(name, book, time, start, end, color=None):
    return Scene(name, int(book), int(time), int(start), int(end), int(color) if color else None)
        
def import_from_file():
    filename = FILENAME
    filetype = FILENAME.split(".")[-1]
    return import_from_csv(filename) if filetype == 'csv' else import_from_xls(filename)
    
def import_from_csv(filename):
    scenes = []
    with open(filename) as f:
        for line in f:
            scene_data = line.strip('\n').split(",")
            scene = create_scene(*scene_data)
            scenes.append(scene)
    return scenes

def import_from_xls(file_name):
    scenes = []
    try:
        wb = load_workbook(filename = file_name)
    except FileNotFoundError as e:
        print("Could not find file {}, please check your settings at the top of this page and try again.".format(file_name))
    
    try:
        ws = wb[WORKSHEET]
    except KeyError as e:
        print("Could not find worksheet {}. Please specify one of {}".format(WORKSHEET, ", ".join(wb.sheetnames)))
        ws = wb[wb.sheetnames[0]]
    
    for row in ws.iter_rows():
        scene = create_scene(*[cell.internal_value for cell in row])
        scenes.append(scene)
    return scenes
    

class TextMap():
    def __init__(self, scenes):
        self.scenes = scenes
        self.figs = {}
        self.layout = self.create_layout()
        self.sort_scenes()
        self.add_bars()
    
    def sort_scenes(self):
        self.scenes.sort(key=lambda scene: (scene.book, scene.time**-1, scene.start))

    def create_layout(self):
        if HEIGHT and WIDTH:
            layout = go.Layout(
                autosize=False,
                width=WIDTH,
                height=HEIGHT
            )
        else:
            layout = go.Layout(autosize=True)
        return layout
        
    def add_bars(self):
        for scene in self.scenes:
            scene.label = next(LABELS)
            bar = go.Bar(x = [scene.end - scene.start],
                        y = [scene.time],
                        base = [scene.start],
                        name = "({}) {}: {}-{}".format(scene.label, scene.name, scene.start, scene.end),
                        orientation = 'h',
                        text = scene.label,
                        textposition = "inside"
                    )
            self.color_bar(scene, bar)
        
            if scene.book in self.figs and isinstance(self.figs[scene.book], go.Figure):
                fig = self.figs[scene.book]
            
            else:
                fig = go.Figure(layout=self.layout)
                fig.update_layout(
                    barmode='relative', 
                    title_text=BOOK_MAP[scene.book],
                    xaxis_title = X_AXIS_TITLE,
                    yaxis = dict(
                        tickmode = 'array',
                        tickvals = list(Y_AXIS.keys()),
                        ticktext = list(Y_AXIS.values())
                    )
                )
                self.figs[scene.book] = fig
            
            fig.add_trace(bar)
    
    def color_bar(self, scene, bar):
        if not scene.color in COLOR_MAP:
            color_list=list(COLOR_MAP.values())
            scene.color = ord(scene.label)%len(color_list)
            bar.marker.color = color_list[scene.color]
            return
        
        bar.marker.color = COLOR_MAP[scene.color]        

    def show(self):
        for fig in self.figs.values():
            fig.show()
            
scenes = import_from_file()
tm = TextMap(scenes)
tm.show()